In [59]:
import time 
import os
import sys
import time
import argparse
import pandas as pd
import numpy as np
import requests
from urllib.parse import quote
from urllib.parse import unquote
from selenium.webdriver.common.keys import Keys 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

In [60]:
def get_url_id(driver, url):
    driver.get(url)
    time.sleep(2)
    # Switch to Next iFrame
    _iframe = driver.find_element(By.TAG_NAME,"iframe")
    driver.switch_to.frame(_iframe)
    time.sleep(3)

    page_id = driver.find_element(By.XPATH, "/html/body/div/div[3]/div/div[2]").text.split('/ ')[1]
    page_id = int(page_id)

    src = driver.find_element(By.XPATH, "/html/body/div/div[4]/div/div[2]/img")
    baseUrl = src.get_attribute('baseURI')
    middle = "/"+ baseUrl.split('/')[-2] + '.pdf_2.jpg'
    baseUrl = baseUrl.replace("index","docs")
    _url = baseUrl.replace(".html",middle)
    
    return _url, page_id

In [61]:
def download_cm(path, date, url, size):
    for i in range(size-1):
        nurl = f'{url}{i+2}.jpg'
        response = requests.get(nurl)
        if response.status_code == 200:
            filename = path + f'/工商時報{date}_{i+1}.jpg'
            with open(filename, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to download image")
def download_cmA(path, date, url, size):
    for i in range(size-1):
        nurl = f'{url}{i+2}.jpg'
        response = requests.get(nurl)
        if response.status_code == 200:
            filename = path + f'/工商時報{date}_A{i+1}.jpg'
            with open(filename, 'wb') as file:
                file.write(response.content)
        else:
            print("Failed to download image")

In [62]:
def download_files(driver, url): 
    _url, url_id = get_url_id(driver, url)
    date = _url.split('/')[-1].split('_')[0]
    url = _url.split('pdf_')[0] + 'pdf_'
    path = f'./工商時報{date}'
    if not os.path.exists(path):
        os.mkdir(path)

    type = _url.split('_')[-2].split('.')[0]
    if type == "CM":
        download_cm(path, date, url, url_id )
    elif type == 'MO' or type == 'ML'or type == 'MN':
        download_cmA(path, date, url, url_id )

In [63]:
def get_all_link(driver):
    output = []
    url = "https://readers.ctee.com.tw/paper"
    driver.get(url)
    btn = driver.find_element(By.XPATH, '/html/body/div[1]/main/div/div/div[1]/div/div/div/div[3]/a')
    btn.click()
    links = driver.find_elements(By.CLASS_NAME, 'paperlink')
    for a in links:
        elem = a.find_element(By.TAG_NAME, 'a')
        _link = elem.get_attribute('href')
        print(_link)
        output.append(_link)
    
    return output

In [64]:
def login(driver, my_pass):
    url = 'https://member.ctee.com.tw/Account/Login'
    driver.get(url)
    driver.maximize_window()
    id = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div[1]/div/div/div/div[1]/section/form/div[1]/div/input")
    _pass = driver.find_element(By.XPATH, "/html/body/div[1]/main/div/div/div/div[1]/div/div/div/div[1]/section/form/div[2]/div/input")
    id.send_keys("travisho0014@gmail.com")
    _pass.send_keys(my_pass)
    button = driver.find_element(By.XPATH,'/html/body/div[1]/main/div/div/div/div[1]/div/div/div/div[1]/section/form/div[4]/div/input')
    button.click()

In [65]:
options = webdriver.ChromeOptions()
#options.add_argument('headless')
driver = webdriver.Chrome(options=options)

In [66]:
login(driver,"Ho20080814")

In [67]:
res = get_all_link(driver)

https://readers.ctee.com.tw/paper?f=20240122_CM&hh=b2017d5d67ca96ba5868cbe9366ec8e4&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240122_MO&hh=e30832272100464747c3f73ec53797bb&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240121_CM&hh=ae502df08037630416e5570ed55259a8&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240121_MN&hh=580dd3fe1a6baa55ad0ab1715126d944&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240120_CM&hh=d3861e63199e22dc2f115a3583afb2ef&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240120_ML&hh=1609bc25716694bfa7fbf4845ad3c201&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240119_CM&hh=ad22205abd879e576824062af66d7d40&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240118_CM&hh=a17ce5ea741720e8076ba82fda2e7bac&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240117_CM&hh=307594f002475a1a72515d45e18acb6d&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240116_CM&hh=586778bc46edd852b778bcf9c6f1f0e8&h=1112&w=2028
https://readers.ctee

In [68]:
file_path = './cteeList.txt'

for r in res:
    r = r + "\n"
    with open(file_path, 'a') as file:
        file.write(r)

In [36]:
res

['https://readers.ctee.com.tw/paper?f=20240116_CM&hh=586778bc46edd852b778bcf9c6f1f0e8&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240115_CM&hh=260594e7704b3657aca46af1b4e30c9c&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240115_MO&hh=bb85b9172998387ff2c0d053b7cd7b67&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240114_CM&hh=97096fc2cefb1752ca83762cfe9e1436&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240114_MN&hh=3ecf2d23c53af4f29f4a0d19ee60b9ba&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240113_CM&hh=ec2435e2135cedaae3eeed6e5c61e3f7&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240113_ML&hh=49b9dddac200652e4c2d4568dbe53490&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240112_CM&hh=1dd556618bf4530bde0f56a0de012efa&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240111_CM&hh=8083232a338124cf917a4a4ea8b1bf77&h=1155&w=2028',
 'https://readers.ctee.com.tw/paper?f=20240110_CM&hh=7665a545c879a7f84eeafc9b0ab98

In [70]:
for x in res[0:7]:
    print(x)
    download_files(driver, x)
    time.sleep(5)

https://readers.ctee.com.tw/paper?f=20240122_CM&hh=b2017d5d67ca96ba5868cbe9366ec8e4&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240122_MO&hh=e30832272100464747c3f73ec53797bb&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240121_CM&hh=ae502df08037630416e5570ed55259a8&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240121_MN&hh=580dd3fe1a6baa55ad0ab1715126d944&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240120_CM&hh=d3861e63199e22dc2f115a3583afb2ef&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240120_ML&hh=1609bc25716694bfa7fbf4845ad3c201&h=1112&w=2028
https://readers.ctee.com.tw/paper?f=20240119_CM&hh=ad22205abd879e576824062af66d7d40&h=1112&w=2028


In [58]:
url = "https://readers.ctee.com.tw/paper?f=20240118_CM&hh=a17ce5ea741720e8076ba82fda2e7bac&h=1155&w=2028"
download_files(driver, url)